In [51]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from folium import GeoJson
from tqdm import tqdm
import numpy as np
from shapely.strtree import STRtree


In [7]:
gdf = gpd.read_file("../resources/gadm_level2.gpkg")
print(gdf.tail())

/opt/anaconda3/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: This version of GeoPackage user_version=0x000028A0 (10400, v1.4.0) on '../resources/gadm_level2.gpkg' may only be partially supported
  return ogr_read(


           GID_2        NAME_2    GID_1              NAME_1 GID_0    NAME_0  \
48128  ZWE.9.5_2  Gwanda Urban  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48129  ZWE.9.6_2        Insiza  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48130  ZWE.9.7_2        Mangwe  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48131  ZWE.9.8_2        Matobo  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48132  ZWE.9.9_2      Plumtree  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   

                                                geometry  
48128  MULTIPOLYGON (((29.00187 -20.95896, 29.00164 -...  
48129  MULTIPOLYGON (((29.35744 -21.02481, 29.35821 -...  
48130  MULTIPOLYGON (((27.81436 -21.19368, 27.81452 -...  
48131  MULTIPOLYGON (((28.45053 -21.64326, 28.44812 -...  
48132  MULTIPOLYGON (((27.83201 -20.57665, 27.82436 -...  


In [8]:
df_images = pd.read_csv("../resources/mp16_combined.csv");
print(df_images.head());

                 IMG_ID        AUTHOR        LAT         LON  S3_Label  \
0  92_17_5276763594.jpg  42441750@N03  38.685568 -109.532951       1.0   
1  0d_ce_6392770405.jpg  68149505@N00  34.933793  103.692741       0.0   
2  2a_88_5268406683.jpg  84867026@N00  39.983433  -75.243301       0.0   
3  82_be_2515710583.jpg  75292316@N00  39.306094  -84.379291       1.0   
4  03_05_9498368699.jpg  61068860@N00   9.186625  123.581597       1.0   

   S16_Label  S365_Label   Prob_indoor  Prob_natural  Prob_urban  \
0        7.0       289.0  1.739840e-04      0.897409    0.102417   
1        1.0       122.0  9.968868e-01      0.000578    0.002535   
2        0.0       128.0  7.201538e-01      0.034871    0.244975   
3        6.0       145.0  9.050690e-05      0.516982    0.482927   
4        8.0        36.0  9.902391e-07      0.999983    0.000016   

  neighbourhood          city               county         state  \
0           NaN           NaN         Grand County          Utah   
1         

In [9]:
geometry = [Point(xy) for xy in zip(df_images['LON'], df_images['LAT'])]
gdf_images = gpd.GeoDataFrame(df_images, geometry=geometry, crs="EPSG:4326")

joined = gpd.sjoin(gdf_images, gdf, how='left', predicate='within')

image_counts = joined.groupby(joined.index_right).size()

gdf['image_count'] = gdf.index.map(image_counts).fillna(0).astype(int)

print(gdf[['GID_2', 'NAME_2', 'image_count']].head(20))

   GID_2 NAME_2  image_count
0                        464
1                          0
2                          0
3                          2
4                        523
5                         26
6                        129
7                        729
8                          0
9                         19
10                        22
11                        67
12                      1109
13                        39
14                         1
15                         0
16                        90
17                       287
18                      2590
19                        52


In [56]:
def merge_small_cells_fast(gdf, min_count=10):
    gdf = gdf.copy()

    # Keep arrays for speed
    geom = gdf.geometry.values
    counts = gdf["image_count"].values
    countries = gdf["GID_0"].values
    index_map = gdf.index.to_numpy()  # track "alive" indices

    count = 0

    while True:
        count+=1
        print(count)
        small_mask = counts < min_count
        if not small_mask.any():
            break
        if count > 4:
            break

        # Build STRtree only once per iteration (fast!)
        tree = STRtree(geom)
        
        # Mapping: small_id → target_id
        merge_map = {}
        # Mapping: target_id → list of geometry indices to union
        groups = {}

        small_indices = np.where(small_mask)[0]

        for si in small_indices:
            small_geom = geom[si]
            if small_geom is None:
                continue

            # R-tree candidate search
            candidate_ids = tree.query(small_geom)
            candidate_ids = [ci for ci in candidate_ids if ci != si]

            if not candidate_ids:
                continue

            # Filter by same country
            same_country = [
                ci for ci in candidate_ids 
                if countries[ci] == countries[si]
            ]
            if not same_country:
                continue

            # Filter by intersects (touches unsafe)
            valid = [
                ci for ci in same_country
                if geom[ci].intersects(small_geom)
            ]
            if not valid:
                continue

            # Select best neighbor (highest image_count)
            best = max(valid, key=lambda ci: counts[ci])

            merge_map[si] = best

            if best not in groups:
                groups[best] = [best]  # start with the target’s own geom
            groups[best].append(si)

        if not merge_map:
            # Nothing to merge = done
            break

        # Apply unions
        for target, members in groups.items():
            new_geom = unary_union([geom[i] for i in members])
            new_count = sum(counts[i] for i in members)

            # Update arrays
            geom[target] = new_geom
            counts[target] = new_count

        # Drop merged-away polygons
        drop_ids = list(merge_map.keys())
        keep_mask = np.ones(len(geom), dtype=bool)
        keep_mask[drop_ids] = False

        geom = geom[keep_mask]
        counts = counts[keep_mask]
        countries = countries[keep_mask]
        index_map = index_map[keep_mask]

    # Reconstruct final GeoDataFrame
    out = gpd.GeoDataFrame(
        {
            "image_count": counts,
            "GID_0": countries
        },
        geometry=geom,
        index=index_map,
        crs=gdf.crs
    )

    return out

In [57]:
merged_gdf = merge_small_cells_fast(gdf)

1
2


In [59]:
merged_gdf.to_file("../resources/gadm_merged.gpkg", driver="GPKG")